In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e8/sample_submission.csv
/kaggle/input/playground-series-s5e8/train.csv
/kaggle/input/playground-series-s5e8/test.csv


In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [3]:
train = pd.read_csv('/kaggle/input/playground-series-s5e8/train.csv')
test = pd.read_csv('/kaggle/input/playground-series-s5e8/test.csv')

In [4]:
train

,id,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,0,42,technician,married,secondary,no,7,no,no,cellular,25,aug,117,3,-1,0,unknown,0
1,1,38,blue-collar,married,secondary,no,514,no,no,unknown,18,jun,185,1,-1,0,unknown,0
2,2,36,blue-collar,married,secondary,no,602,yes,no,unknown,14,may,111,2,-1,0,unknown,0
3,3,27,student,single,secondary,no,34,yes,no,unknown,28,may,10,2,-1,0,unknown,0
4,4,26,technician,married,secondary,no,889,yes,no,cellular,3,feb,902,1,-1,0,unknown,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
749995,749995,29,services,single,secondary,no,1282,no,yes,unknown,4,jul,1006,2,-1,0,unknown,1
749996,749996,69,retired,divorced,tertiary,no,631,no,no,cellular,19,aug,87,1,-1,0,unknown,0
749997,749997,50,blue-collar,married,secondary,no,217,yes,no,cellular,17,apr,113,1,-1,0,unknown,0
749998,749998,32,technician,married,secondary,no,-274,no,no,cellular,26,aug,108,6,-1,0,unknown,0


In [5]:
# encoding categorical data
le = LabelEncoder()
train_en = train.copy()
for col in train_en[['job','marital','education','default','housing','loan','contact','month','poutcome']]:
    train_en[col]= le.fit_transform(train_en[col])
train_en

,id,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,0,42,9,1,1,0,7,0,0,0,25,1,117,3,-1,0,3,0
1,1,38,1,1,1,0,514,0,0,2,18,6,185,1,-1,0,3,0
2,2,36,1,1,1,0,602,1,0,2,14,8,111,2,-1,0,3,0
3,3,27,8,2,1,0,34,1,0,2,28,8,10,2,-1,0,3,0
4,4,26,9,1,1,0,889,1,0,0,3,3,902,1,-1,0,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
749995,749995,29,7,2,1,0,1282,0,1,2,4,5,1006,2,-1,0,3,1
749996,749996,69,5,0,2,0,631,0,0,0,19,1,87,1,-1,0,3,0
749997,749997,50,1,1,1,0,217,1,0,0,17,0,113,1,-1,0,3,0
749998,749998,32,9,1,1,0,-274,0,0,0,26,1,108,6,-1,0,3,0


In [6]:
#test validation split
from sklearn.model_selection import train_test_split
x_train = train_en.drop(columns = ['id','y'])
y_train = train_en['y']
xt,xv,yt,yv = train_test_split(x_train,y_train,test_size = 0.2,random_state = 42)

In [7]:
model = LogisticRegression()
model.fit(xt, yt)
prob = model.predict(xv)

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [8]:
prob

array([0, 0, 0, ..., 0, 0, 0])

In [9]:
from sklearn.metrics import mean_squared_log_error
# --- RMSLE function
def rmsle(y_true, y_pred):
    y_pred = np.maximum(0, y_pred)
    return np.sqrt(mean_squared_log_error(y_true, y_pred))

In [10]:
print(rmsle(yv,prob))

0.22272218609067193


In [11]:
test_en = test.copy()
test_en = test_en.drop(columns = ['id'])
for col in test_en[['job','marital','education','default','housing','loan','contact','month','poutcome']]:
    test_en[col]= le.fit_transform(test_en[col])

In [12]:
result = model.predict_proba(test_en)

In [13]:
result

array([[0.95289735, 0.04710265],
       [0.6441619 , 0.3558381 ],
       [0.96808917, 0.03191083],
       ...,
       [0.68411626, 0.31588374],
       [0.98665302, 0.01334698],
       [0.85514033, 0.14485967]])

In [14]:
submission = pd.DataFrame()
submission['id'] = test['id']
submission['y'] = result[:,1]

In [15]:
submission.to_csv("submission.csv", index=False)